# Part 3: Intermediate PySpark Patterns

**Goal**: Master the essential patterns between basics and optimization

**What You'll Learn**:
1. Window Functions - The analytics powerhouse
2. Caching & Persistence - Reusing computed results
3. Partitioning Fundamentals - Data layout control
4. explain() & Query Plans - Understanding Spark's execution
5. Multi-table Joins - Real-world pipeline patterns
6. Complex Data Types - Arrays, structs, and nested data

**Why This Matters**: These patterns appear in 80% of production PySpark jobs!

In [ ]:
# Setup: Import required libraries
from pyspark.sql.functions import *
from pyspark.sql.window import Window
from pyspark.sql.types import *

# Load TPC-H datasets
orders_df = spark.table("samples.tpch.orders") 
customers_df = spark.table("samples.tpch.customer")
lineitem_df = spark.table("samples.tpch.lineitem")
nation_df = spark.table("samples.tpch.nation")

print(f"✓ Orders: {orders_df.count():,} rows")
print(f"✓ Customers: {customers_df.count():,} rows")
print(f"✓ Lineitem: {lineitem_df.count():,} rows")


## Module 3.1: Window Functions - Analytics Powerhouse

**The Problem**: Need to rank, compare, or calculate rolling metrics within groups

**Window Functions**: Perform calculations across rows *related to* the current row
- Unlike `groupBy`, you keep all original rows
- Essential for: ranking, running totals, moving averages, lag/lead


In [ ]:
### Use Case 1: Rank top customers by total spending

# Calculate total spending per customer
customer_spending = orders_df.join(
    customers_df,
    orders_df.o_custkey == customers_df.c_custkey
).groupBy("c_custkey", "c_name", "c_mktsegment").agg(
    sum("o_totalprice").alias("total_spent"),
    count("o_orderkey").alias("order_count")
)

# Define window: partition by market segment, order by spending
window_spec = Window.partitionBy("c_mktsegment").orderBy(desc("total_spent"))

# Apply window functions
ranked_customers = customer_spending.withColumn(
    "rank_in_segment", rank().over(window_spec)
).withColumn(
    "row_number_in_segment", row_number().over(window_spec)
)

# Show top 3 customers per segment
ranked_customers.filter(col("rank_in_segment") <= 3) \
    .orderBy("c_mktsegment", "rank_in_segment") \
    .select("c_mktsegment", "c_name", "total_spent", "order_count", "rank_in_segment") \
    .show(15)

In [ ]:
### Use Case 2: Running Total - Cumulative revenue over time

# Aggregate orders by date
daily_revenue = orders_df.groupBy("o_orderdate").agg(
    sum("o_totalprice").alias("daily_revenue")
).orderBy("o_orderdate")

# Define window: order by date (unbounded preceding to current row)
running_window = Window.orderBy("o_orderdate").rowsBetween(Window.unboundedPreceding, Window.currentRow)

# Calculate running total
cumulative_revenue = daily_revenue.withColumn(
    "cumulative_revenue",
    sum("daily_revenue").over(running_window)
)

cumulative_revenue.show(10)


In [ ]:
### Use Case 3: lag/lead - Compare with previous/next values

# For each customer, compare order with their previous order
customer_orders = orders_df.select(
    "o_custkey", "o_orderkey", "o_orderdate", "o_totalprice"
).filter(col("o_custkey") <= 10)  # Sample for clarity

# Window: partition by customer, order by date
customer_window = Window.partitionBy("o_custkey").orderBy("o_orderdate")

order_comparison = customer_orders.withColumn(
    "previous_order_price", lag("o_totalprice", 1).over(customer_window)
).withColumn(
    "next_order_price", lead("o_totalprice", 1).over(customer_window)
).withColumn(
    "price_change_from_previous",
    col("o_totalprice") - col("previous_order_price")
)

order_comparison.orderBy("o_custkey", "o_orderdate").show(15)


## Module 3.2: Caching & Persistence [DON'T RUN IN DATABRICKS - DOESN"T WORK IN SERVERLESS]

**The Problem**: Recomputing expensive transformations multiple times wastes resources

**Solution**: Cache intermediate results that you'll reuse

**When to Cache**:
- ✅ You'll use the DataFrame multiple times
- ✅ The computation is expensive
- ✅ You have enough memory
- ❌ Don't cache if you only use it once


In [ ]:
### Scenario: Expensive join used multiple times

# Create an expensive aggregation (will be reused)
customer_metrics = orders_df.join(
    customers_df,
    orders_df.o_custkey == customers_df.c_custkey
).groupBy(
    "c_custkey", "c_name", "c_mktsegment", "c_nationkey"
).agg(
    sum("o_totalprice").alias("total_spent"),
    count("o_orderkey").alias("order_count"),
    avg("o_totalprice").alias("avg_order_value")
)

# WITHOUT caching - compute twice (inefficient)
print("=== WITHOUT CACHING ===")
print(f"High spenders: {customer_metrics.filter(col('total_spent') > 500000).count()}")
print(f"Frequent buyers: {customer_metrics.filter(col('order_count') > 30).count()}")
print("⚠️ The join and aggregation ran TWICE!\n")

# WITH caching - compute once, reuse
print("=== WITH CACHING ===")
#customer_metrics.cache()  # Mark for caching

# First action materializes the cache
print(f"High spenders: {customer_metrics.filter(col('total_spent') > 500000).count()}")
print(f"Frequent buyers: {customer_metrics.filter(col('order_count') > 30).count()}")
print("✅ Second query used cached data!")

In [ ]:
### persist() - More control than cache()

from pyspark import StorageLevel

# Different storage levels
print("💾 Storage Levels:")
print("   MEMORY_ONLY - Fast, but can be evicted")
print("   MEMORY_AND_DISK - Spills to disk if needed")
print("   DISK_ONLY - Slower, but saves memory")
print("   MEMORY_ONLY_SER - Serialized (saves space)\n")

# cache() is shorthand for persist(StorageLevel.MEMORY_ONLY)
# df_memory_and_disk = customer_metrics.persist(StorageLevel.MEMORY_AND_DISK)

print("✅ Persisted with MEMORY_AND_DISK")
print("\n💡 Best Practice: Use MEMORY_AND_DISK for production")

In [ ]:
### Clean up: unpersist when done

# customer_metrics.unpersist()
print("✅ Cache cleared - memory freed")

print("\n🎯 Caching Checklist:")
print("   1. Will I use this DataFrame 2+ times? → cache()")
print("   2. Is computation expensive? → cache()")
print("   3. Do I have enough memory? → Check Spark UI")
print("   4. Done with cached data? → unpersist()")


## Module 3.3: Partitioning Fundamentals [DON'T RUN - DOESN'T WORK IN DATABRICKS WITH SERVERLESS]

**What are Partitions?**: Logical chunks of your data that can be processed in parallel

**Why It Matters**:
- More partitions = More parallelism (up to a point)
- Too few partitions = Wasted resources
- Too many partitions = Overhead

**Rule of Thumb**: 2-4 partitions per CPU core


In [ ]:
### Check current partitions

print(f"Orders DataFrame: {orders_df.rdd.getNumPartitions()} partitions")
print(f"Customers DataFrame: {customers_df.rdd.getNumPartitions()} partitions")

# After a shuffle operation (like groupBy), partitions change
grouped = orders_df.groupBy("o_orderstatus").count()
print(f"After groupBy: {grouped.rdd.getNumPartitions()} partitions")
print(f"   (Default: spark.sql.shuffle.partitions = {spark.conf.get('spark.sql.shuffle.partitions')})")


In [ ]:
### repartition() vs coalesce()

# repartition() - Full shuffle, can increase or decrease
repartitioned = orders_df.repartition(50)
print(f"After repartition(50): {repartitioned.rdd.getNumPartitions()} partitions")
print("   → Full shuffle (expensive, but balanced)\n")

# coalesce() - No shuffle, can only decrease
coalesced = repartitioned.coalesce(10)
print(f"After coalesce(10): {coalesced.rdd.getNumPartitions()} partitions")
print("   → No shuffle (fast, may be unbalanced)\n")

print("💡 When to use:")
print("   repartition() - Need more partitions OR rebalance")
print("   coalesce() - Need fewer partitions (like before writing)")

In [ ]:
### Partition by column - Smart data layout

# Repartition by a key you'll filter on frequently
partitioned_by_status = orders_df.repartition(10, "o_orderstatus")

print(f"Partitions: {partitioned_by_status.rdd.getNumPartitions()}")
print("\n💡 Benefits of partitioning by column:")
print("   • Filters on that column scan fewer partitions")
print("   • Joins on that column avoid shuffles")
print("   • Write to disk with partition column for pruning")


In [ ]:
### Saving with partitioning - Disk layout optimization

username = spark.sql("SELECT current_user()").collect()[0][0]
clean_username = username.split('@')[0].replace('.', '_').replace('-', '_')
table_name = f"{clean_username}_partitioned_orders"

# Save partitioned by order status
orders_df.write.format("delta") \
    .mode("overwrite") \
    .partitionBy("o_orderstatus") \
    .saveAsTable(table_name)

print(f"✅ Saved to: {table_name}")
print("\n💡 Partition pruning in action:")
print("   • Query: WHERE o_orderstatus = 'F'")
print("   • Spark only reads 'F' partition folder!")
print("   • Result: Much faster queries on partitioned columns")


## Module 3.4: explain() & Understanding Query Plans

**Why Learn This?**: To understand what Spark is *actually* doing with your code

**Query Plan Stages**:
1. **Parsed Logical Plan** - Your code translated
2. **Analyzed Logical Plan** - With schema info
3. **Optimized Logical Plan** - After Catalyst optimizations
4. **Physical Plan** - Actual execution strategy


In [ ]:
### Simple explain() - Physical plan only

query = orders_df.filter(col("o_orderstatus") == "F") \
    .select("o_orderkey", "o_totalprice") \
    .groupBy().avg("o_totalprice")

print("=== Physical Plan ===")
query.explain()

print("\n💡 Read bottom-to-top:")
print("   1. FileScan (read data)")
print("   2. Filter (WHERE clause)")
print("   3. Project (SELECT columns)")
print("   4. HashAggregate (GROUP BY)")


In [ ]:
### Extended explain - See all stages

print("=== All Query Plan Stages ===")
query.explain("extended")

print("\n🔍 Notice:")
print("   • Catalyst optimizer simplified the query")
print("   • Filter pushed down close to scan (efficient!)")
print("   • Only needed columns are read")


In [ ]:
### Spot the difference: Join strategies

# Small table join (might use broadcast)
small_join = orders_df.limit(100).join(
    nation_df,
    orders_df.o_custkey == nation_df.n_nationkey  # Wrong join for demo
)

print("=== Small Table Join ===")
small_join.explain()

print("\n🔍 Look for:")
print("   • BroadcastHashJoin - Fast (no shuffle on small side)")
print("   • SortMergeJoin - Slower (shuffle both sides)")


## Module 3.5: Multi-table Joins - Real Pipeline Pattern

**Real World**: You rarely join just 2 tables

**Best Practices**:
1. Filter early (before joins)
2. Select only needed columns (before joins)
3. Join large tables first, dimension tables last
4. Consider broadcast for small dimensions


In [ ]:
### Scenario: Customer → Orders → LineItems → Analysis

# Step 1: Filter and select early
customers_filtered = customers_df.select(
    "c_custkey", "c_name", "c_mktsegment", "c_nationkey"
).filter(col("c_mktsegment") == "BUILDING")

orders_filtered = orders_df.select(
    "o_orderkey", "o_custkey", "o_orderdate", "o_totalprice"
).filter(year(col("o_orderdate")) >= 1995)

lineitems_filtered = lineitem_df.select(
    "l_orderkey", "l_quantity", "l_extendedprice", "l_discount"
)

# Step 2: Join in logical order
enriched_data = customers_filtered \
    .join(orders_filtered, customers_filtered.c_custkey == orders_filtered.o_custkey) \
    .join(lineitems_filtered, orders_filtered.o_orderkey == lineitems_filtered.l_orderkey) \
    .join(nation_df, customers_filtered.c_nationkey == nation_df.n_nationkey)

# Step 3: Aggregate
result = enriched_data.groupBy("n_name", "c_name").agg(
    sum("l_extendedprice").alias("total_revenue"),
    sum("l_quantity").alias("total_quantity"),
    countDistinct("o_orderkey").alias("order_count")
).orderBy(desc("total_revenue"))

result.show(10)

print("\n✅ Best practices applied:")
print("   • Filtered BEFORE joins (less data to shuffle)")
print("   • Selected only needed columns")
print("   • Joined dimension table (nation) last")


In [ ]:
### Check the execution plan

result.explain()

print("\n🔍 Notice in plan:")
print("   • Filters pushed down (applied during scan)")
print("   • Column pruning (only selected columns read)")
print("   • Join order optimized by Catalyst")


## Module 3.6: Complex Data Types

**Real World**: Data is messy and nested

**Common Types**:
- **Arrays**: Lists of values `[1, 2, 3]`
- **Structs**: Nested objects `{name: "John", age: 30}`
- **Maps**: Key-value pairs `{"key1": "value1"}`


In [ ]:
### Working with Arrays

# Create an array column: collect orders per customer
customer_order_arrays = orders_df.groupBy("o_custkey").agg(
    collect_list("o_orderkey").alias("order_keys"),
    collect_list("o_totalprice").alias("order_prices"),
    count("o_orderkey").alias("order_count")
).filter(col("order_count") >= 20)

customer_order_arrays.show(5, truncate=False)

# Array operations
array_ops = customer_order_arrays.withColumn(
    "max_order_price", array_max("order_prices")
).withColumn(
    "min_order_price", array_min("order_prices")
).withColumn(
    "array_size", size("order_keys")
)

array_ops.select("o_custkey", "order_count", "max_order_price", "min_order_price").show(10)


In [ ]:
### Explode - Arrays to rows

# Transform array into separate rows
exploded = customer_order_arrays.select(
    "o_custkey",
    explode("order_keys").alias("order_key")  # Each array element becomes a row
)

print(f"Original rows: {customer_order_arrays.count()}")
print(f"After explode: {exploded.count()}")
exploded.show(10)

print("\n💡 explode() converts: [1,2,3] → 3 separate rows")


In [ ]:
### Working with Structs (nested objects)

# Create a struct column
with_struct = orders_df.withColumn(
    "order_summary",
    struct(
        col("o_orderkey").alias("key"),
        col("o_totalprice").alias("price"),
        col("o_orderstatus").alias("status")
    )
).select("o_custkey", "order_summary")

with_struct.show(5, truncate=False)

# Access struct fields with dot notation
struct_access = with_struct.select(
    "o_custkey",
    col("order_summary.key").alias("order_key"),
    col("order_summary.price").alias("order_price")
)

struct_access.show(5)

print("\n💡 Structs are useful for:")
print("   • Grouping related fields")
print("   • Working with JSON data")
print("   • Nested schema operations")
